In [35]:
import pandas as pd
loan=pd.read_csv('output.csv')
pd.set_option('display.max_columns', None)

In [36]:
X=loan.drop(columns=['loan_status','loan_status_grouped','emp_title','purpose','title','issue_d','grade'])
y=loan['loan_status_grouped']

In [37]:
X.head()

,annual_inc,delinq_2yrs,dti,emp_length,funded_amnt,home_ownership,inq_last_6mths,installment,int_rate,loan_amnt,mths_since_last_delinq,pub_rec,sub_grade,term,total_acc,total_pymnt,acc_now_delinq,log_annual_inc
0,24000.0,0.0,27.65,10+ years,5000.0,RENT,1.0,162.87,10.65,5000.0,0.0,0.0,B2,36 months,9.0,5861.071414,0.0,10.085809
1,30000.0,0.0,1.00,< 1 year,2500.0,RENT,5.0,59.83,15.27,2500.0,0.0,0.0,C4,60 months,4.0,1008.710000,0.0,10.308953
2,12252.0,0.0,8.72,10+ years,2400.0,RENT,2.0,84.33,15.96,2400.0,0.0,0.0,C5,36 months,10.0,3003.653644,0.0,9.413444
3,49200.0,0.0,20.00,10+ years,10000.0,RENT,1.0,339.31,13.49,10000.0,35.0,0.0,C1,36 months,37.0,12226.302212,0.0,10.803649
4,36000.0,0.0,11.20,3 years,5000.0,RENT,3.0,156.46,7.90,5000.0,0.0,0.0,A4,36 months,12.0,5631.377753,0.0,10.491274


In [38]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans

def sub_grades_encoding(x):
    grade_mapping = {'A': 7, 'B': 6, 'C': 5, 'D': 4, 'E': 3, 'F': 2, 'G': 1}
    number_mapping = {'1': 0.8, '2': 0.6, '3': 0.4, '4': 0.2, '5': 0.0}
    
    val = grade_mapping.get(x[0], 0) + number_mapping.get(x[1], 0)
    return val

In [39]:
X['sub_grade']= X['sub_grade'].apply(sub_grades_encoding)

In [40]:
#Separate numerical and non-numerical columns
ordinal_columns = ['emp_length']
categorical_columns = ['home_ownership', 'term']
numerical_columns = X.select_dtypes(include=['float64']).columns

# Apply Label Encoding to ordinal columns before splitting
le = LabelEncoder()
for col in ordinal_columns:
    X[col] = le.fit_transform(X[col])

# Apply OneHotEncoding to categorical columns before splitting
X_encoded = pd.get_dummies(X, columns=categorical_columns)

# Now perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Ensure the train and test sets have the same structure (dummies already handled)
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

# Apply scaling to numerical columns
scaler = StandardScaler()
X_train[numerical_columns] = scaler.fit_transform(X_train[numerical_columns])
X_test[numerical_columns] = scaler.transform(X_test[numerical_columns])

In [41]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train,y_train)
pred_knn = knn.predict(X_test)
print("Classification Report :")
print(     classification_report(y_test,pred_knn))

Classification Report :
              precision    recall  f1-score   support

         0.0       0.92      0.55      0.69     10809
         1.0       0.89      0.99      0.94     40517

    accuracy                           0.90     51326
   macro avg       0.91      0.77      0.81     51326
weighted avg       0.90      0.90      0.88     51326

